# Waterflow: A Water Quality Prediction System

Dans ce projet, nous allons développer un système de prédiction de la qualité de l'eau en utilisant des modèles de machine learning. Nous allons utiliser MLflow pour le suivi des expériences et la gestion des modèles.

## Installation des dépendances
```bash
pip install -r requirements.txt
```
## Importation des bibliothèques nécessaires

In [1]:
import mlflow

Instantiation de `mlflow` à l'aide de la commande suivante dans le terminal :
```bash
mlflow server --host 127.0.0.1 --port 5000
```

Puis accès à l'interface web de MLflow via `http://localhost:5000`.

In [2]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("experiment_water_quality")

<Experiment: artifact_location='mlflow-artifacts:/316023407685107800', creation_time=1751444283180, experiment_id='316023407685107800', last_update_time=1751444283180, lifecycle_stage='active', name='experiment_water_quality', tags={}>